# Introducción al webscraping

## Introducción

### ¿De qué se compone una web?

[HTML](https://developer.mozilla.org/es/docs/Web/HTML), [CSS](https://developer.mozilla.org/es/docs/Web/CSS) y [Javascript](https://developer.mozilla.org/es/docs/Web/JavaScript) 

### [PETICIONES HTTP](https://developer.mozilla.org/es/docs/Web/HTTP/Methods)

GET, POST, PUT, DELETE, PATCH, ... 


### [HTML](https://developer.mozilla.org/es/docs/Web/HTML)

ETIQUETAS, ATRIBUTOS, ...

### [SELECTORES CSS](https://www.w3schools.com/cssref/css_selectors.php)

. # > +

In [1]:
import bs4
bs4.__version__

'4.12.2'

## Importamos librerías

En primer lugar importamos las librerías sys, csv y statistics, que vienen por defecto en Python.

In [1]:
import sys 
import csv
import statistics

Comprobamos ahora si tenemos instaladas las librerías Beautiful Soup, Request y Pandas. Si no es así las instalamos

In [2]:
%%capture
if 'BeautifulSoup' not in sys.modules:
    !{sys.executable} -m pip install beautifulsoup4
    
if 'requests' not in sys.modules:
    !{sys.executable} -m pip install requests
    
if 'requests' not in sys.modules:
    !{sys.executable} -m pip install pandas

Importamos ahora Beautiful Soup, Requests y Pandas

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Funciones auxiliares

Ponemos un par de funciones auxiliares que usaremos un poco más adelante

In [4]:
# Para exportar listas a archivos csv
def export_list_to_csv(rows, columns=None, filename="output", delimiter=";"):

    if type(rows[0]) == str:
        rows = [[row] for row in rows]

    if columns:
        assert len(columns) == len(rows[0]), "There should be the same number of columns and rows elements"
 
    with open(f"{filename}.csv", 'w') as f:
        write = csv.writer(f, delimiter=delimiter)

        if columns:
            write.writerow(columns)

        write.writerows(rows)

In [5]:
# Para dividir una lista en trozos más pequeños
def chunks(lst, n):
    return [lst[i:i + n] for i in range(0, len(lst), n)]

## Bloque 1 - Extrayendo los titulares de un periódico

In [6]:
WEB_BLOQUE1 = 'https://www.elconfidencial.com'

En primer lugar usamos la librería requests para bajarnos el HTML de la web que queremos descargar

In [7]:
x = requests.get(WEB_BLOQUE1)

Podemos ver el [status code de nuestra petición web](https://developer.mozilla.org/es/docs/Web/HTTP/Status). Si el status code es 200, es que todo ha ido bien

In [8]:
x.status_code

200

Para consultar el código HTML descargado usamos la propiedad text

In [9]:
x.text

'<!DOCTYPE html><html lang="es"><head><script type="application/json" id="EC_hosts">{"name": "El Confidencial","id": "1","enviroment": "production","host": "www.elconfidencial.com","api": "api.elconfidencial.com","secure": "secure.elconfidencial.com","image": "","useridentity": "useridentity.elconfidencial.com"}</script><meta http-equiv="Content-Type" content="text/html; charset=utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=edge"/><!--[if IE 8]><script src="/javascript/v2/plugins/html5-shim/html5shiv-printshiv.js"></script><![endif]--><title>El Confidencial - El diario de los lectores influyentes</title><meta name="msapplication-config" content="none"/><meta name="verify-v1" content="zaF1JNxc7UQaY6aOtg5/g3MYrxKOipO3XWaUNnhtBgQ=" /><meta name="google-site-verification" content="e4XcvLCkuMSwwudpCP7hG5oi7Odb9VJ4U9207IpyvBk" /><link rel="canonical" href="https://www.elconfidencial.com/" /><meta name="twitter:dnt" content="on" /><meta name="lang" content="es"  /><meta name="title" 

El siguiente paso es parsear el documento. El parseo es el proceso de analizar un texto para identificar su estructura y poder extraer información de ella. Existen diferentes parseadores que pueden usarse. En nuestro caso usaremos html.parser

In [10]:
parsed_document = BeautifulSoup(x.text, "html.parser")

Tras revisar la web, vemos que los artículos del periódico comparten la clase titleArticleEditable. Utilizamos el selector CSS de clases (el .) para seleccionarlos.

In [11]:
all_titles_nodes = parsed_document.css.select(".titleArticleEditable")

Comprobamos que estamos extrayendo el titular

In [12]:
all_titles_nodes[0].text

'        El Gobierno se abre a que Rajoy y otros dirigentes del PP vayan al Congreso por espiar a Cataluña    '

Extraemos ahora todos los artículos, reemplazamos los saltos de línea por espacios en blanco y quitamos los espacios en blanco al principio y al final

In [13]:
all_titles = [article.text.replace("\n", " ").lstrip().rstrip() for article in all_titles_nodes]

Veamos ahora todos los titulares

In [14]:
all_titles

['El Gobierno se abre a que Rajoy y otros dirigentes del PP vayan al Congreso por espiar a Cataluña',
 'Moncloa desclasifica los autos que avalaron el pinchazo a Aragonès',
 'Generación porno: el sexo extremo, nuevo tutor de niños y jóvenes',
 'Sumar y Podemos piden al Ejecutivo que vete a BlackRock la entrada en Naturgy',
 'Cuerpo analiza si la gestora debe pedir el plácet para tomar el 20% de la gasista',
 'Santander pide precio a las Big Four para el mayor contrato de la historia',
 'Moncloa iguala el tabaco calentado al tradicional y prohíbe aromatizantes',
 'Sánchez da galones a Puente, Saiz, Redondo y Hereu en la ejecutiva',
 "Feijóo impone tres plenos al mes en el Senado y lo convierte en su 'fortín'",
 'Ayuso alerta de que "ETA está ganando el relato" y pide llevar a Bildu al TS',
 'El PNV pide a Junts "rebajar los decibelios" del debate migratorio',
 "Mazón pone el ventilador al Botànic: denuncia 713 M en pagos a dedo y anuncia una auditoría 'forensic'",
 'Cataluña prohíbe lle

## Bloque 2 - Descargar los datos diarios de la bolsa

In [15]:
WEB_BLOQUE2 = "https://www.expansion.com/mercados/cotizaciones/indices/igbolsamadrid_I.MA.html"

Nos bajamos el HTML de la web donde tenemos los datos de la bolsa

In [16]:
x = requests.get(WEB_BLOQUE2)

Parseamos el documento

In [17]:
parsed_cotiz = BeautifulSoup(x.text, "html.parser")

Extraemos los nombres de las columnas

In [18]:
all_header_nodes = parsed_cotiz.css.select("#listado_valores th")
all_header = [article.text.replace("\n", " ").lstrip().rstrip() for article in all_header_nodes if article.text]

In [19]:
all_header

['Valor',
 'Último',
 'Var. %',
 'Var.',
 'Ac. % año',
 'Máx.',
 'Mín.',
 'Vol.',
 'Capit.',
 'Hora',
 '']

Extraemos los datos

In [20]:
all_values_nodes = parsed_cotiz.css.select("#listado_valores td")
all_values = [article.text for article in all_values_nodes]

In [21]:
all_values

['ACCIONA',
 '122,350',
 '-4,23',
 '-5,40',
 '-8,21',
 '126,650',
 '122,100',
 '71.279',
 '6.712',
 '16:32',
 '',
 'ACCIONA ENER',
 '23,980',
 '-3,46',
 '-0,86',
 '-14,60',
 '24,620',
 '23,920',
 '231.352',
 '7.895',
 '16:32',
 '',
 'ACERINOX',
 '10,135',
 '-2,55',
 '-0,26',
 '-4,88',
 '10,400',
 '10,125',
 '544.593',
 '2.742',
 '16:29',
 '',
 'ACS',
 '39,620',
 '-1,59',
 '-0,64',
 '-1,34',
 '40,080',
 '39,500',
 '172.874',
 '11.021',
 '16:31',
 '',
 'ADOLFO DOMÍNGUEZ',
 '4,840',
 '-1,63',
 '-0,08',
 '-3,20',
 '4,900',
 '4,840',
 '133',
 '45',
 '12:42',
 '',
 'AEDAS HOMES',
 '18,260',
 '-1,51',
 '-0,28',
 '0,22',
 '18,640',
 '18,260',
 '4.511',
 '855',
 '16:00',
 '',
 'AENA',
 '166,300',
 '-1,48',
 '-2,50',
 '1,34',
 '167,550',
 '164,100',
 '42.131',
 '24.945',
 '16:31',
 '',
 'AIRBUS',
 '147,940',
 '-0,14',
 '-0,20',
 '5,40',
 '148,400',
 '146,920',
 '798',
 '116.293',
 '16:03',
 '',
 'AIRTIFICIAL',
 '0,145',
 '-2,03',
 '-0,00',
 '12,02',
 '0,149',
 '0,142',
 '3.446.195',
 '193',
 '16

Dividimos los datos por filas

In [22]:
all_values = chunks(all_values, 11)

In [23]:
all_values

[['ACCIONA',
  '122,350',
  '-4,23',
  '-5,40',
  '-8,21',
  '126,650',
  '122,100',
  '71.279',
  '6.712',
  '16:32',
  ''],
 ['ACCIONA ENER',
  '23,980',
  '-3,46',
  '-0,86',
  '-14,60',
  '24,620',
  '23,920',
  '231.352',
  '7.895',
  '16:32',
  ''],
 ['ACERINOX',
  '10,135',
  '-2,55',
  '-0,26',
  '-4,88',
  '10,400',
  '10,125',
  '544.593',
  '2.742',
  '16:29',
  ''],
 ['ACS',
  '39,620',
  '-1,59',
  '-0,64',
  '-1,34',
  '40,080',
  '39,500',
  '172.874',
  '11.021',
  '16:31',
  ''],
 ['ADOLFO DOMÍNGUEZ',
  '4,840',
  '-1,63',
  '-0,08',
  '-3,20',
  '4,900',
  '4,840',
  '133',
  '45',
  '12:42',
  ''],
 ['AEDAS HOMES',
  '18,260',
  '-1,51',
  '-0,28',
  '0,22',
  '18,640',
  '18,260',
  '4.511',
  '855',
  '16:00',
  ''],
 ['AENA',
  '166,300',
  '-1,48',
  '-2,50',
  '1,34',
  '167,550',
  '164,100',
  '42.131',
  '24.945',
  '16:31',
  ''],
 ['AIRBUS',
  '147,940',
  '-0,14',
  '-0,20',
  '5,40',
  '148,400',
  '146,920',
  '798',
  '116.293',
  '16:03',
  ''],
 ['AIR

In [24]:
pd.DataFrame(data=all_values, columns=all_header).iloc[:,:10].set_index('Valor')

,Último,Var. %,Var.,Ac. % año,Máx.,Mín.,Vol.,Capit.,Hora
Valor,,,,,,,,,
ACCIONA,"122,350","-4,23","-5,40","-8,21","126,650","122,100",71.279,6.712,16:32
ACCIONA ENER,"23,980","-3,46","-0,86","-14,60","24,620","23,920",231.352,7.895,16:32
ACERINOX,"10,135","-2,55","-0,26","-4,88","10,400","10,125",544.593,2.742,16:29
ACS,"39,620","-1,59","-0,64","-1,34","40,080","39,500",172.874,11.021,16:31
ADOLFO DOMÍNGUEZ,"4,840","-1,63","-0,08","-3,20","4,900","4,840",133,45,12:42
...,...,...,...,...,...,...,...,...,...
UNICAJA BANCO,"0,898","-1,21","-0,01","0,90","0,906","0,893",5.084.168,2.384,16:31
URBAS,"0,005","-2,04","-0,00","11,63","0,005","0,005",49.642.888,67,16:15
VIDRALA,"90,300","-1,31","-1,20","-3,73","90,500","89,700",9.595,2.913,16:15


## Bloque 3 - ¿Quién utiliza más palabras en sus canciones?

Localizamos en primer lugar una página web con letras de artistas que sea fácilmente parseable y le damos al opción de 

In [25]:
WEB_BLOQUE3 = "https://www.letras.com"
GRUPO = "sfdk"

Extraemos ahora todos los links con canciones de un artista

In [26]:
x = requests.get(WEB_BLOQUE3 + "/" + GRUPO)

In [27]:
WEB_BLOQUE3 + "/" + GRUPO

'https://www.letras.com/sfdk'

In [28]:
parsed_song_links = BeautifulSoup(x.text, "html.parser")
all_links_nodes = parsed_song_links.css.select("#cnt-artist-songlist a.songList-table-songName")
all_links = [[article.text, f"{WEB_BLOQUE3}{article.attrs['href']}"] for article in all_links_nodes]
all_links

[['2005', 'https://www.letras.com/sfdk/367297/'],
 ['3 Hombres Y Un Destino', 'https://www.letras.com/sfdk/367298/'],
 ['35 Rimas', 'https://www.letras.com/sfdk/610726/'],
 ['41.008 (nivel Pino Montano)', 'https://www.letras.com/sfdk/367299/'],
 ['A Donde Van', 'https://www.letras.com/sfdk/367300/'],
 ['A mi no me lo cuentes', 'https://www.letras.com/sfdk/610727/'],
 ['Abuchea!!', 'https://www.letras.com/sfdk/367301/'],
 ['Ahora Yo Les Traigo El Sabor', 'https://www.letras.com/sfdk/367302/'],
 ['Al Filo', 'https://www.letras.com/sfdk/367303/'],
 ['Bailes de Salón (con Juaninacka)', 'https://www.letras.com/sfdk/367304/'],
 ['Baobab (part. Escoberito)',
  'https://www.letras.com/sfdk/baobab-part-escoberito/'],
 ['Crossover', 'https://www.letras.com/sfdk/367305/'],
 ['Cruzcampo', 'https://www.letras.com/sfdk/610728/'],
 ['Cuarta Sinfonia', 'https://www.letras.com/sfdk/286089/'],
 ['Defectos y Taras', 'https://www.letras.com/sfdk/defectos-y-taras/'],
 ['Del Barrio Para El Barrio (part. El 

Con esto y la función que definimos al principio del documento podríamos exportar si quisiéramos todas las letras y sus nombres a un archivo

In [29]:
all_links[74]

['Un Pobre Con Dinero', 'https://www.letras.com/sfdk/un-pobre-con-dinero/']

In [30]:
export_list_to_csv(all_links, ['cancion', 'direccion'], GRUPO)

Extraigamos ahora la letra de una canción utilizando su dirección web

In [31]:
y = requests.get(all_links[74][1])
parsed_song = BeautifulSoup(y.text, "html.parser")

Seleccionemos los párrafos con las letras

In [32]:
song_lyrics_nodes = parsed_song.css.select(".lyric p")

Reemplacemos los saltos de línea de HTML (\</br>) con espacios en blanco

In [33]:
for el in song_lyrics_nodes:
    [br.replace_with("\n").text for br in el.select("br")]

Transformemos ahora la canción de un objeto parseado de beautiful soup a texto

In [34]:
song_lyrics_nodes = [p.text for p in song_lyrics_nodes]
song_lyrics_nodes = "\n".join(song_lyrics_nodes)

Comprobemos como queda la canción

In [35]:
print(song_lyrics_nodes)

Vuelvo a tener seis añitos de edad
Mi madre recién separá
Los vecinos tuvieron pa mi buenos gestos na ma'
Vacaciones invitaá'
Estate despierto tienes que trabajar
Era otro tiempo, otra mentalidad
Aún me despierto de noche jartito e sudar
Soñando a mi madre es lo má'
Recibitos que pagar
Papelitos Panamá, tu casa es la mía, primo, sí
Solo si vengo a currar
"Trabaja pa'l niño pijo", ya lo dijo mi mamá
Mi abuela cuidó un cortijo, y antes de ella su mamá
Un paseo mirando al mar
Casas que no pueo pagar
Y en el caso que pudiese
¿Quién coño las va a limpiar?
Sueña un niño en la barriá
Que un día lloverá el maná
Y el pantanal no se estaba tan mal
No se estaba tan mal
No se estaba tan mal
No se estaba tan mal
No se estaba tan mal
No se estaba tan mal
No sé lo que quiero ejercer
Pero sí lo que no quiero ser
He nacido sin miedo a perder
Lo que gané, lo aposté
Me da seis frutos cada dos por tres, yo apenas disfruto, me da mucho corte
Me dejé lo mejor para el postre
Ya que nunca jamás me postré
¿Qué

Con esto podemos contar el número de palabras distintas que hay en la letra

In [36]:
len(set(song_lyrics_nodes.replace("\n", " ").split(" ")))

222

Definamos ahora una función que haga todos estos pasos

In [37]:
def get_lyrics(url):

    y = requests.get(url)
    parsed_song = BeautifulSoup(y.text, "html.parser")

    song_lyrics_nodes = parsed_song.css.select(".lyric p")

    for el in song_lyrics_nodes:
        [br.replace_with("\n").text for br in el.select("br")]

    song_lyrics_nodes = [p.text for p in song_lyrics_nodes]
    song_lyrics_nodes = "\n".join(song_lyrics_nodes)

    palabras_diferentes = set(song_lyrics_nodes.replace("\n", " ").split(" "))

    return len(palabras_diferentes)

Podemos ahora ver cuantas palabras distintas tiene cada canción del artista

In [38]:
canciones = {all_links[i][0] : get_lyrics(all_links[i][1]) for i in range(len(all_links))}
canciones

{'2005': 224,
 '3 Hombres Y Un Destino': 435,
 '35 Rimas': 115,
 '41.008 (nivel Pino Montano)': 286,
 'A Donde Van': 262,
 'A mi no me lo cuentes': 491,
 'Abuchea!!': 266,
 'Ahora Yo Les Traigo El Sabor': 285,
 'Al Filo': 268,
 'Bailes de Salón (con Juaninacka)': 418,
 'Baobab (part. Escoberito)': 230,
 'Crossover': 454,
 'Cruzcampo': 306,
 'Cuarta Sinfonia': 382,
 'Defectos y Taras': 231,
 'Del Barrio Para El Barrio (part. El Límite y Karvoh)': 188,
 'Desafío Total': 488,
 'Desde Los Chiqueros': 218,
 'Desde Pinomontano Hasta Los Vestuarios': 492,
 'Desde Sevilla Hasta Tu Tierra': 406,
 'Despedida Y Cierre (que Os Follen': 265,
 'Después de...': 338,
 'Diana Fácil': 308,
 'Dolar Mas Euro (con Tote King)': 475,
 'Donde Esta Wifly': 235,
 'Duelo de Vikingos': 193,
 'El Blues Del Condenado (part. Lia Kali)': 197,
 'El despreocupado': 182,
 'El Diablo de Alma Buena': 276,
 'El Doctor': 292,
 'El Liricista En El Tejado': 377,
 'El Niño Güei': 365,
 'El Perro Anda Suelto': 225,
 'En La Oscu

Podemos ahora calcular el número medio de palabras distintas de cada artista

In [39]:
print(statistics.mean([canciones[key] for key in canciones.keys()]))

293.9012345679012
